# Calculate the original, low, and high pass filtered SST anomalies

1. Model names and paths
2. Read the data
3. Calculate the original SST anomalies
4. Calculate the low pass filtered SST anomalies
5. Calculate the high pass filtered SST anomalies
6. Save the data



In [1]:
import xarray as xr
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import numpy as np
from scipy.stats import linregress
from scipy import stats
from itertools import cycle
import cftime
from cmip_postprocess import *  
import warnings
warnings.filterwarnings('ignore')

/global/homes/y/yanxia/.conda/envs/climate/lib/python3.10/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


2024-10-22 14:49:10.093939


In [2]:
# Configuration
paths = {
    'cmip': '../data/raw/', # downloaded from Pangeo
    'MMF': '../data/raw/E3SM-MMF_1950-2015_Regrid180x360.nc', # E3SM-MMF simulation output
    'E3SM': '../data/raw/E3SM_1950-2015_regrid180x360.nc', # E3SM simulation output
    'ersst': '../data/raw/ersst.v5.185401-201512.nc', # ERSST observation
    'Hadley_SST2': '../data/raw/HadISST.0-360.nc' # HadISST observation
}

rolling_times = {
    'decadal': 121, # 10 years
}


In [9]:
def get_input_names(path, model_file='source_id.txt'):
    with open(f"{path}{model_file}", "r") as f:
        models = [line.strip() for line in f]
    return models, [f"{path}{model}" for model in models]


def get_data_type(model):
    if model in ['ERSST']:
        return 3
    elif model in ['HadISST']:
        return 1
    else:
        return 2

# write a function to calculate the ssta 
# 1. Read the data
# 2. Calculate the SST anomalies
# 3. Detrend the data
# 4. optional: rolling mean, no rolling mean for original ssta, decadal for lpf 
def cal_ssta(models, files, rolling_time=None):
    ssta = {}
    for model, file in zip(models, files):
        print('Start processing: {0}'.format(model))
        ssta[model] = read_data(file, data_type=get_data_type(model), model=model)
        ssta[model], _ = ano_norm_t(ssta[model], rolling_time=rolling_time)
        ssta[model] = detrend_dim(ssta[model], 'time')

        decadal = rolling_times['decadal']
        time_slice = slice(int(decadal/2), -int(decadal/2))
        ssta[model] = ssta[model].isel(time=time_slice)

    return ssta

In [ ]:
# Include the observation data and simulation data: HadISST, ERSST, E3SM-MMF, E3SMv2
models, model_files = get_input_names(paths['cmip'])
models.extend(['E3SM-MMF', 'E3SMv2', 'HadISST', 'ERSST'])
model_files.extend([paths['MMF'], paths['E3SM'], paths['Hadley_SST2'], paths['ersst']])

In [ ]:
# Calculate the original SST anomalies
orig_ssta = cal_ssta(models, model_files, rolling_time=None)

In [ ]:
# Calculate the low pass filtered SST anomalies
lpf_ssta = cal_ssta(models, model_files, rolling_time=rolling_times['decadal'])

In [ ]:
# Calculate the high pass filtered SST anomalies as the difference between the original and low pass filtered SST anomalies
hpf_ssta = {}
for model in models:
    hpf_ssta[model] = orig_ssta[model] - lpf_ssta[model]

In [ ]:
# Save the data to pickle files
save_datasets_to_pickle(lpf_ssta, '../data/processed/lpf_ssta.pkl')
save_datasets_to_pickle(orig_ssta, '../data/processed/orig_ssta.pkl')
save_datasets_to_pickle(hpf_ssta, '../data/processed/hpf_ssta.pkl')
